# FONDECYT
## Codigo para la descarga de poligonos, desde coordenadas UTM
### Manuel Silva manuel.silva@pucv.cl
### Laboratorio de Robótica y Visión, EIE-PUCV

## Celda 1: Importación de Módulos

In [ ]:
# Instalar modulos necesarios
%pip install eodag==2.12.1
%pip install Shapely==2.0.3

In [ ]:
# Importamos los módulos necesarios para ejecutar el código
from eodag import EODataAccessGateway, setup_logging
from shapely.geometry import Polygon

## Celda 2: Configuración de Logging

In [ ]:
# Configuramos el registro para mostrar información detallada durante la ejecución
setup_logging(2)

## Celda 3: Creación de la instancia EODataAccessGateway

In [ ]:
# Creamos una instancia de EODataAccessGateway para manejar las operaciones de datos
dag = EODataAccessGateway()

## Celda 4: Configuración del proveedor de datos

In [ ]:
# Establecemos el proveedor preferido para los datos de satélite
dag.set_preferred_provider("cop_dataspace")

## Celda 5: Entrada de coordenadas

In [ ]:
# Pedimos al usuario que ingrese las coordenadas manualmente
# Formato sacado de https://geojson.io/ 

coordinates = [
    [-71.61585623944886, -33.04283802364129],
    [-71.61585623944886, -33.04719451892468],
    [-71.60413501869623, -33.04719451892468],
    [-71.60413501869623, -33.04283802364129],
    [-71.61585623944886, -33.04283802364129]  # Cerrar el polígono repitiendo el primer punto
]

## Celda 6: Creación del polígono

In [ ]:
# Creamos un polígono a partir de las coordenadas ingresadas
roi_polygon = Polygon(coordinates)

## Celda 7: Configuración de criterios de búsqueda

In [ ]:
# Configuramos los criterios de búsqueda para los datos de satélite
search_criteria = {
    "productType": "S2_MSI_L2A",
    "start": "2022-01-01",
    "end": "2023-03-01",
    "geom": roi_polygon
}

## Celda 8: Búsqueda de productos

In [ ]:
# Realizamos la búsqueda de productos que coincidan con los criterios
search_result = dag.search(**search_criteria)


## Celda 9: Resultados de la búsqueda

In [ ]:
# Imprimimos el tipo y contenido del resultado de búsqueda para depuración
print(type(search_result))
print(search_result)


## Celda 10: Manejo de resultados

In [ ]:
# Procesamos los resultados de la búsqueda, descargando los productos disponibles
if search_result:
    if isinstance(search_result, tuple):
        products, estimated_total = search_result
        if products:
            for product in products:
                if hasattr(product, 'location') and product.location.startswith("file:/"):
                    print("Local product detected. Download skipped")
                else:
                    download_path = dag.download(product)
                    print(f"Descargado: {download_path}")
        else:
            print("No products found.")
    else:
        # Otro manejo si se espera otro tipo de objeto
        pass
else:
    print("No results returned.")